# Tutorial FFT 2D sequential

In this tutorial, we present how to use fluidfft to perform fft2d in sequential. It is rather straightforward...

In [1]:
import numpy as np
from fluidfft.fft2d import methods_seq
from fluidfft import import_fft_class

In [2]:
methods_seq

['fft2d.with_fftw1d', 'fft2d.with_fftw2d', 'fft2d.with_cufft']

We import a class and instantiate it:

In [3]:
cls = import_fft_class('fft2d.with_fftw2d')

In [4]:
o = cls(16, 48)

Let's have a look at the attribute of this objects. They are actually all methods:

In [5]:
[name for name in dir(o) if not name.startswith('__')]

['compute_energy_from_K',
 'compute_energy_from_X',
 'fft',
 'fft_as_arg',
 'gather_Xspace',
 'get_is_transposed',
 'get_k_adim_loc',
 'get_local_size_X',
 'get_seq_indices_first_K',
 'get_seq_indices_first_X',
 'get_shapeK_loc',
 'get_shapeK_seq',
 'get_shapeX_loc',
 'get_shapeX_seq',
 'get_short_name',
 'get_x_adim_loc',
 'ifft',
 'ifft_as_arg',
 'run_benchs',
 'run_tests',
 'scatter_Xspace',
 'sum_wavenumbers']

Let's run a test and benchmark the fft and ifft functions directly from C++.

In [6]:
o.run_tests()

1

In [7]:
t1, t2 = o.run_benchs()
't_fft = {} s; t_ifft = {} s'.format(t1, t2)

't_fft = 1.3100000000000002e-05 s; t_ifft = 4.9e-06 s'

Let's understand how the data is stored:

In [8]:
o.get_is_transposed()

False

which means that for this class, in Fourier space, the data is not transposed...

Now we can get the non dimensional wavenumber in the first and second dimensions:

In [9]:
k0, k1 = o.get_k_adim_loc()
print('k0:', k0)
print('k1:', k1)

k0: [ 0  1  2  3  4  5  6  7  8 -7 -6 -5 -4 -3 -2 -1]
k1: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


and check that the shapes of the array in one process are the same than in sequential (we are in sequential, there is only one process):

In [10]:
assert o.get_shapeX_loc() == o.get_shapeX_seq()
assert o.get_shapeK_loc() == o.get_shapeK_seq()

Now, let's compute a fast Fourier transform:

In [11]:
a = np.ones(o.get_shapeX_loc())
a_fft = o.fft(a)

If we already have the array where we want to put the result we can do:

In [12]:
a = np.ones(o.get_shapeX_loc())
a_fft = np.empty(o.get_shapeK_loc(), dtype=np.complex128)
o.fft_as_arg(a, a_fft)

<MemoryView of 'ndarray' at 0x7f73d40741f0>

And finally for the inverse Fourier transform:

In [13]:
o.ifft_as_arg(a_fft, a)

<MemoryView of 'ndarray' at 0x7f73d4074398>

In [14]:
a = o.ifft(a_fft)